<a href="https://colab.research.google.com/github/dbizzaro/Minesweeper/blob/main/Minesweeper_with_SATsolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minesweeper

A project by **Davide Bizzaro**, mat. **203851**, for the course *Knowledge and data mining*, at Unipd 2021/2022.

In [1]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.5 MB/s 
     |████████████████████████████████| 3.4 MB 26.9 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=838290407dcfd0e964af4eab75287124ea8fd11d7e22f92bced7d4fe34249482
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [2]:
from pysat.card import CardEnc
from pysat.solvers import Minisat22
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.special import binom

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Class for game handling

The following class comprises all that concerns the game but not the solver.

If you don't know the game, see [en.wikipedia.org/wiki/Minesweeper_(video_game)](en.wikipedia.org/wiki/Minesweeper_(video_game)).

In [3]:
class Board:
  '''
  Objects of this class summarize game board (e.g. where the mines are placed) and game status (e.g. which cells have been revealed, and what number is in them).
  
  Attributes:
  - n_rows: number of rows
  - n_cols: number of columns
  - n_mines: number of mines
  - board: array (of shape  n_rows x n_cols) filled with integers:
      - -1 represents general covered cells
      - -2 represent covered cells where the solver thinks there are mines
      - non-negative numbers represents revealed cells, where the number is the number of mines in the adjacent cells
  - mines: set of positions -of the form (row,col)- where the mines lie. Its kept private (so it's sure the solver has no direct access to it) and initialized randomly
  '''
  
  def __init__(self, n_rows, n_cols, n_mines, seed=None):
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.n_mines = n_mines
    self.board = np.full((n_rows, n_cols), -1, dtype='short')
    self.__mines = set()
    np.random.seed(seed)
    placed = 0
    while (placed < n_mines):
      row = np.random.randint(0,n_rows)
      col = np.random.randint(0,n_cols)
      if ((row,col) not in self.__mines):
        self.__mines.add((row,col))
        placed += 1

  def adjacent_cells(self, position, only_among_still_covered=True):
    '''
    returns the list of the covered cells adjacent to that given as argument.
    If only_among_still_covered is set to False, then it returns the list of all the adjacent cells, not just the covered ones
    '''
    adj = []
    for x in range(position[0]-1, position[0]+2):
        for y in range(position[1]-1, position[1]+2):
            if (x>=0 and y>=0 and x<self.n_rows and y<self.n_cols and (x, y) != position):
              if ((not only_among_still_covered) or self.board[x,y]<0):
                adj.append((x,y))
    return adj

  def n_adjacent_mines(self, position):
    '''
    returns the number of mines in cells adjacent to the given position
    '''
    adj = self.adjacent_cells(position)
    n = 0
    for pos in adj:
      if (pos in self.__mines):
        n += 1
    return n

  def n_observed_cells(self):
    '''
    returns the number of visited cells (thus containing the non negative number of neighboring mines)
    '''
    return (self.board >= 0).sum()

  def check_victory(self):
    '''
    returns true if you have won the game (i.e. observed all the cells without mines)
    '''
    if (self.n_observed_cells() == self.n_rows * self.n_cols - self.n_mines):
      return True
    return False

  def randomly_fill_in(self, n_observed, seed=None):
    '''
    fills in n_observed cells (without mines), at random position, with the correct number. 
    it should be used just once per board, when you do not want to start with an all-covered board.
    '''
    np.random.seed(seed)
    observed = 0
    assert 0 <= n_observed <= self.n_rows * self.n_cols - self.n_mines, "you are requiring to fill in a wrong number of cells"
    while (observed < n_observed):
      row = np.random.randint(0,self.n_rows)
      col = np.random.randint(0,self.n_cols)
      if (self.board[row,col] < 0 and ((row,col) not in self.__mines)):
        self.board[row,col] = self.n_adjacent_mines((row, col))
        observed += 1
  
  def show(self, display_mines=False, explosion_at=None):
    '''
    method for printing the board, with or without the indication of the mines
    it is also possible to indicate on the board the position of an exploded mine that caused a game over
    '''
    df = pd.DataFrame(self.board, dtype='object')
    df = df.replace(-1, '').replace(-2, '🚩')
    if display_mines:
      for pos in self.__mines:
        df.iloc[pos] = '💣'
      if (explosion_at in self.__mines):
        df.iloc[explosion_at] = '💥'
    df.style
    display(df)

  def still_covered_cells(self, separate_cells_with_an_uncovered_neighbor=False):
    '''
    returns the list of all the covered cells (those without numbers appearing on the displayed board)
    if separate_cells_with_an_uncovered_neighbor is set to true, then it returns instead two lists:
      - the list of covered cells adjacent to at least one revealed cell
      - the list of all the other covered cells (those not adjacent to any revealed cell)
    ''' # I could have written two separate methods, but it is not important
    covered_cells_with_an_uncovered_neighbor = []
    covered_cells_without_an_uncovered_neighbor = []
    covered_cells = []
    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if (self.board[row,col] < 0):
          if not separate_cells_with_an_uncovered_neighbor:
            covered_cells.append((row,col))
          elif len(self.adjacent_cells((row, col), True))!=len(self.adjacent_cells((row, col), False)):
            covered_cells_with_an_uncovered_neighbor.append((row, col))
          else:
            covered_cells_without_an_uncovered_neighbor.append((row, col))
    if not separate_cells_with_an_uncovered_neighbor:
      return covered_cells
    return covered_cells_with_an_uncovered_neighbor, covered_cells_without_an_uncovered_neighbor

  def reveal_position(self, position, verbosity=2):
    '''
    this method represent the response of the game at the choice of a cell to reveal.
    if the cell contains a mine, then it returns -2, indicating game over
    if the cell is sound and does not contain a mine, then it returns the non-negative number of mines in adjacent cells, which is also set to be shown on the board from now on
    - use verbosity = 0 when you don't want any output at all
    - use verbosity > 0 when you want to output what is going on (but without displaying the board if it is not game over)
    '''
    n_adj_mines = self.n_adjacent_mines(position)
    if (position in self.__mines):
      if verbosity > 0:
        print("BOOM! Exploded mine in position {}.". format(position))
        self.show(True, position)
      return -2
    if (self.board[position] < 0):
      self.board[position] = n_adj_mines
      if verbosity > 0:
        print("Revealed position {}, which has {} neighboring mines.".format(position, n_adj_mines))
      return n_adj_mines
    print('Invalid position')
    return -3


maximum number clauses from single cardinality constrain: $\binom{8}{5}+ \binom{8}{3}=112$

### Functions for game solving

We use MiniSat 2.2 in order to produce an algorithm for solving the game.

We consider propositional variables $X_{ij}$ saying "in position $(i,j)$ there is a mine". As we are using DIMACS format for CNF clauses, they are represented by a natural number from 1 to the total number of cells. 

The axioms we need to provide are: 

1.   the variables corresponding to the cells with a visible number are false;
2.   for every cell with a visible number k, exactly k of the variables corresponding to the adjacent cells are true;
3.   exactly n of the variables are true, where n is the total number of mines specified by the game.

We encoded the cardinality constrains of point 2 in a very simple way: for every cell with number $k$ and set of covered adjacent cells $S$, with cardinality $n$, we built
\begin{equation} ⋀_{I ⊂ S, \ |I|=n-k+1}⋁_{(i,j) \in I} X_{ij} \quad \quad \quad \text{(at least k)}\end{equation}
and
\begin{equation} ⋀_{I ⊂ S, \ |I|=k+1}⋁_{(i,j) \in I} \neg X_{ij} \quad \quad \quad \text{(at most k)}\end{equation}

This was because in this case the number $n$ of adjacent cells is limited by $8$, so the total number of clauses does not grow too much. 

But the cardinality constrain of point 3 cannot be treated in the same way because it would need too many clauses. Therefore we used a cardinality encoder given in the *pysat* module (that reduces the number of needed clauses by introducing auxiliary variables). See [pysathq.github.io/docs/html/api/card.html](https://pysathq.github.io/docs/html/api/card.html) for the relative documentation.

In [4]:
def variable_mine_at(n_cols, position):
  '''
  produce a different number for each cell, representing a propositional variable saying "a mine is there".
  '''
  return 1 + position[0] * n_cols + position[1]

def proposition_exactly_k_neighboring_mines(board, position, k):
  '''
  given a position and the number k of adjacent mines, it encodes the proposition saying "there are exactly k mines among the adjacent cells"
  '''
  adj = board.adjacent_cells(position)
  n = len(adj)
  clauses=[]
  for I in combinations(adj, n-k+1):  # at least k
    clauses.append([variable_mine_at(board.n_cols, i) for i in I])
  for I in combinations(adj, k+1):  # at most k
    clauses.append([-variable_mine_at(board.n_cols, i) for i in I])
  return clauses

def proposition_total_number_of_mines(board):
  '''
  it encodes the proposition saying "there are exactly k mines among all cells, where k is the total number of mines specified by the game"
  '''
  available_positions = board.still_covered_cells()
  variables = [variable_mine_at(board.n_cols, pos) for pos in available_positions]
  n = board.n_rows * board.n_cols
  k = board.n_mines
  cnf = CardEnc.equals(variables, k, top_id=n) # top_id=n means that the cardinality encoder can produce auxiliary varibles starting from n+1
  return cnf.clauses

def generate_propositions(board, satsolver):
  '''
  it feeds the given satsolver with the formula specifying the total number of mines and, 
  if there are already revealed numbers on the board, also the formulas representing all the implied constrains.
  '''
  for row in range(board.n_rows):
    for col in range(board.n_cols):
      k = board.board[row,col]
      if k >=0:
        satsolver.add_clause([-variable_mine_at(board.n_cols, (row,col))])                      # point 1 of the discussion
        satsolver.append_formula(proposition_exactly_k_neighboring_mines(board, (row,col), k))  # point 2 of the discussion
  satsolver.append_formula(proposition_total_number_of_mines(board))                            # point 3 of the discussion

Now we can build functions that, given a position and a satsolver feeded with the axioms previously seen, tells us if the position is safe or not, and if it surely has a mine or not. The first is given by checking if the axioms plus the literal $X_{ij}$ (corresponding to the position) gives UNSAT: if it is the case, then it is valid *not* having a mine in that position, meaning that the position is safe and we can click on confidently. The second is given by checking if the axioms plus the literal $\neg X_{ij}$ gives UNSAT: if it is the case, then it is valid having a mine in that position, meaning that we can confidently say that there is a mine there.

In [5]:
def check_safe_position_at(board, position, satsolver): 
  ''' 
  return True if the satsolver can infer that there is no mine at the given position, False otherwise
  '''
  return not satsolver.solve(assumptions=[variable_mine_at(board.n_cols, position)]) 

def check_mine_at(board, position, satsolver): 
  ''' 
  return True if the satsolver can infer that there is surely a mine at the given position, False otherwise
  '''
  return not satsolver.solve(assumptions=[-variable_mine_at(board.n_cols, position)])

Before constructing the complete algorithm for solving the game, we need a way to handle the case in which we find no safe position, and we want to choose the most probable one, that is the one that allows for more models in which it is safe. We do that using model counting, but in the *pysat* module the only function allowing for that is one generating all models one by one. This is not very efficient, but it is enough for our purposes, thanks to a simple trick.

Our trick is to consider separately the covered cells adjacent to at least one cell with a visible number, which we will call "bounded cells", and the covered cells not adjacent to any cell with a visible number, which we will call "free cells". This is because the only constrain we have on the positions of the mines among the free cell is on the total cardinality, and, fixed that, any disposition is equally likely. This mean that assuming to have exactly $k$ mines among the $n$ free cells, the possible dispositions are simply $\binom{n}{k}$. Therefore, any model for the bounded cells leaving out exactly $k$ mines to be on the free cells gives rise to exactly  $\binom{n}{k}$ different general models. This reduces a lot the enumerations of models we need to perform. 

In the end, first we consider the number of models not having a mine on the first free cell (if there exists one). It should be clear from our discussion that this is equal to the number of models not having a mine on any other specific free cell, so that is why we consider just the first free cell. Again from the previous discussion it follows that such number of models is
\begin{equation} \sum_{k} \binom{n-1}{k} \#SAT_{\text{bounded cells}}(k) \end{equation}

where $n$ is the number of free cells, $\#SAT_{\text{bounded cells}}(k)$ is the number of models for the bounded cells leaving out exactly $k$ mines to be placed on the free cells, and the sum goes from $k=\max(0,\#mines-\#bounded cells)$ to $k=\min(n-1, \#mines)$.
Then, we cycle through the bounded cells, and for each one we compute in a similar way the number of models not having a mine in that position, which is
\begin{equation} \sum_{k} \binom{n}{k} \#SAT_{\text{bounded cells}}(k, position) \end{equation}
where again $n$ is the number of free cells, but now $\#SAT_{\text{bounded cells}}(k, position)$ is the number of models for the bounded cells leaving out exactly $k$ mines to be placed on the free cells and such that in *position* there is not a mine, and the sum goes from $k=\max(0,\#mines-\#bounded cells+1)$ to $k=\min(n, \#mines)$.


**Note**: the function *enum_models* that we are using is built in a way that in order not to find out again a model already found, it negates each model it finds, before looking for another. Therefore, since we do not want exclude permanently those models from possible subsequent calls of the satsolver, we introduce a new auxiliary variable specific for each call of *enum_models*: each call will look for models in which the auxiliary variable is True, and when it is done the variable is forever set to False.

**Note2**: Commented there is some code that computes the number of models by simple enumeration, without using our trick. I used it to check that the function calculated the right numbers of models, and you can de-comment and use it for the same purpose. Notice however that it would be highly inefficient to keep it and not use our trick.

In [6]:
def count_models(satsolver, assumptions):
  '''
  count the number of models satisfying the given assumptions (and the axioms already given to the satsolver)
  '''
  return sum(1 for _ in satsolver.enum_models(assumptions=assumptions))

def most_probable_safe_position(board, satsolver):
  ''' 
  find a still covered position on the board giving the largest number of models not having a mine in that position
  '''
  
  top_id = satsolver.nof_vars()  # see the first note in the text above
  
  def assumption_exactly_k_mines_among_free_cells(k):
    '''
    generate the clauses saying that exactly k mines are placed on free cells
    it does so taking in consideration the note in the text above, 
    and the fact that it is not important where we place the k mines among the free cells, since every model for the free cells is equally likely
    '''
    assumption = [variable_mine_at(board.n_cols, free_cells[i]) for i in range(k)] # at least k
    assumption += [-variable_mine_at(board.n_cols, free_cells[i+k]) for i in range(n_free_cells-k)] # at most k
    assumption += [-i for i in range(top_id+1, satsolver.nof_vars()+1)] + [satsolver.nof_vars()+1] # see the first note in the text above
    return assumption

  #def assumption_note():
  #  assumption = [-i for i in range(top_id+1, satsolver.nof_vars()+1)] + [satsolver.nof_vars()+1]
  #  return assumption

  bounded_cells, free_cells = board.still_covered_cells(separate_cells_with_an_uncovered_neighbor=True)
  n_bounded_cells = len(bounded_cells)
  n_free_cells = len(free_cells)
  counter_n_models = 0
  for n_mines_in_free_cells in range(max(0,board.n_mines-n_bounded_cells), min(n_free_cells-1, board.n_mines)+1):
    assumption = assumption_exactly_k_mines_among_free_cells(n_mines_in_free_cells)
    counter_n_models += count_models(satsolver, assumption) * binom(n_free_cells-1, n_mines_in_free_cells)
  #print("counter (first case)          :", counter_n_models)
  #print("total enumeration (first case):", count_models(satsolver, assumption_note()+[-variable_mine_at(board.n_cols, free_cells[0])]))
  if n_free_cells > 0: 
    maximum = counter_n_models
    best_option = free_cells[0]
  else:
    maximum = 0 
    best_option = None
  for position in bounded_cells:
    counter_n_models = 0
    for n_mines_in_free_cells in range(max(0,board.n_mines-n_bounded_cells+1), min(n_free_cells, board.n_mines)+1):
      assumption = assumption_exactly_k_mines_among_free_cells(n_mines_in_free_cells) + [-variable_mine_at(board.n_cols, position)]
      counter_n_models += count_models(satsolver, assumption) * binom(n_free_cells, n_mines_in_free_cells)
    #print("counter (second case)          :", counter_n_models)
    #print("total enumeration (second case):", count_models(satsolver, assumption_note()+[-variable_mine_at(board.n_cols, position)]))
    if counter_n_models > maximum:
      maximum = counter_n_models
      best_option = position
  satsolver.append_formula([[-i] for i in range(top_id+1, satsolver.nof_vars()+1)]) # see the first note in the text above
  return best_option

It is finally time to write a function that plays the game. First we cycle among bounded cells: if one is found safe, we reveal it and continue; if one is found to have a mine, we take note of it and continue. Then since running the satsolver is computationally expensive and all free cells are equally likely to be safe or to have a mine, we consider just the first one and do for it the same tests as before. 

If we find at least a safe position in all the cycle, then we restart it again, in order to see if the new information we have found out can be used to go on safely. If instead we find no safe position in a whole cycle, it means that we have to take a risk in order to continue (if we have not reached the end of the game). We choose to reveal the cell which is more likely not to have a mine (see the discussion in the text above). Then we restart from the beginning again, until the end of the game.

Note that surely there could be a better way to stack the cells to be tested for safety in a way to minimize (heuristically) the number of calls to the satsolver (which is the most computationally expensive part, after model counting). It is just that for my purposes the efficiency is already fine.

In [37]:
def play_game(board, verbosity=2, n_moves_before_showing_board=1):
  '''
  solve a game given as an object of the class Board, and returns True when it ends with a victory; False otherwise
  - use verbosity = 0 when you don't want any output at all
  - use verbosity = 1 when you just want to output what is going on, without displaying the board if it is not game over or victory
  - use verbosity = 2 when you want also to display the board after every n_moves_before_showing_board steps (default being 1, i.e. after each move)
  '''
  counter_for_showing_board = 1
  n_mines_found = 0
  solver = Minisat22()
  generate_propositions(board, solver)
  game_over_flag = False
  while not game_over_flag:
    safe_positions_found = 0
    bounded_cells, free_cells = board.still_covered_cells(separate_cells_with_an_uncovered_neighbor=True)
    if len(free_cells) > 0: # since free cells are all equally likely, we just need to check if 
      cells_to_try = bounded_cells + [free_cells[0]]
    else:
      cells_to_try = bounded_cells
    for position in cells_to_try:
      if (board.board[position] == -1 and check_mine_at(board, position, solver)): 
        solver.add_clause([variable_mine_at(board.n_cols, position)]) # add clause saying we've found a mine
        board.board[position] = -2
        if verbosity > 0:
          print("Found mine in position {}".format(position))
          if (verbosity > 1 and counter_for_showing_board % n_moves_before_showing_board == 0):
            board.show()
        n_mines_found += 1
        counter_for_showing_board += 1
      if (board.board[position] == -1 and check_safe_position_at(board, position, solver)):  # if position safe, then ...
        n_adj_mines = board.reveal_position(position, verbosity) # ... reveal position ...
        assert n_adj_mines >= 0, "The program said it was safe, but it was not :("
        solver.append_formula(proposition_exactly_k_neighboring_mines(board, position, n_adj_mines))
        solver.add_clause([-variable_mine_at(board.n_cols, position)])
        if (verbosity > 1 and counter_for_showing_board % n_moves_before_showing_board == 0):
            board.show()
        safe_positions_found += 1
        counter_for_showing_board += 1
    if safe_positions_found == 0 and n_mines_found == board.n_mines:
      if verbosity > 0:
        print("The program has won!")
        board.show(True)
      break
    elif safe_positions_found == 0:
      if verbosity > 0:
          print("I'm not sure. Let's hope.")
      position_to_try = most_probable_safe_position(board, solver)
      n_adj_mines = board.reveal_position(position_to_try, verbosity) # ... reveal position ...
      if n_adj_mines < 0:
        game_over_flag = True
        if verbosity > 0:
          print("Game Over: the program has lost.")
      else:
        solver.append_formula(proposition_exactly_k_neighboring_mines(board, position_to_try, n_adj_mines))
        solver.add_clause([-variable_mine_at(board.n_cols, position_to_try)])
        if (verbosity > 1 and counter_for_showing_board % n_moves_before_showing_board == 0):
            board.show()
        counter_for_showing_board += 1
  solver.delete() # free up RAM
  return not game_over_flag

In [8]:
def statistics(n_rows, n_cols, n_mines, sample_size):
  for i in range(sample_size):
    new_board = Board(n_rows, n_cols, n_mines)
    play_game(new_board)

In [38]:
example = Board(4, 4, 6)
example.randomly_fill_in(0)
example.show(True)

,0,1,2,3
0,💣,,💣,💣
1,,,💣,
2,,,,
3,,,💣,💣


In [39]:
play_game(example, 2)

I'm not sure. Let's hope.
BOOM! Exploded mine in position (0, 0).


,0,1,2,3
0,💥,,💣,💣
1,,,💣,
2,,,,
3,,,💣,💣


Game Over: the program has lost.


False

TO DO:


*   verbosity
*   comments
*   statistics

